In [1]:
import os
os.environ['R_HOME']='/opt/miniconda3/envs/tiptox/lib/R'
# %load ../env.py
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline
#%load_ext rpy2.ipython

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>")) # increase jupyter screen width to 100%


import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 
from matplotlib import gridspec
from scipy.interpolate import splev, splrep
import xlwt
import seaborn as sns
from scipy import stats
import rpy2.robjects.packages as rpackages
import seaborn as sns
from statsmodels import robust

TOP = "/home/ishah/ipynb/heprn-tp/"

LIB = TOP+'lib3'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

from db.mongo import *

HCI=openMongo(db='heprn_v1',host='localhost',auth=False)
#DB=openMongo(db='httr_ph1',host='pb.epa.gov')
#DB1=openMongo(db='httr_v1',host='pb.epa.gov')
#MSG=openMongo(db='msigdb_v6',host='pb.epa.gov')
#CMP=openMongo(db='cmap_v2',host='pb.epa.gov',host='localhost')
#from gexp.deseq2 import *
pd.options.display.max_colwidth = 500
pd.set_option('display.precision',2)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns
sns.set(style='whitegrid')

# Goal 

Calculate AEDs using IVIVE functions

# Chemicals

In [3]:
from tk.httk import * 


RAT_LIV_LL = pd.read_excel(DAT_DIR+'toxref-v2.0-pods-rat-liver.xlsx')
CH_CAS0=pd.DataFrame(list(HCI.chemicals.find()),
                    columns=['dsstox_sid','chem_name','casrn'])\
        .rename(columns=dict(chem_name='name'))
CAS_TK=getChemInfo()
CAS0 = set(CH_CAS0.casrn).intersection(RAT_LIV_LL.casrn).intersection(CAS_TK)
CH_CAS1 = CH_CAS0[CH_CAS0.casrn.isin(CAS0)]

/opt/miniconda3/envs/tiptox/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/opt/miniconda3/envs/tiptox/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


# *In vitro* Potency

## HepRn

The HepRn CR analysis is conducted as follows:
* Curve fits are in heprn/heprn-cr-v2b.ipynb and results stored in mongodb://pb.epa.gov/heprn_v1.heprn_cr 
* Curve fit quality is analyzed in heprn-ivive/heprn-conc-response-v1c.ipynb and stored in mongodb://pb.epa.gov/heprn_v1/heprn_hitqual
* The final hits are identified based on quality and the results are stored in 
mongodb://pb.epa.gov/heprn_v1/potency

### Load hits

In [4]:
RES = []

FT_CH = ['MF_dn','DT_up','St_up','NS_dn','Ap_up','CN_dn','St_dn',
         'DT_dn','NS_up','ES_dn','ES_up','LM_up','LM_dn']

Q = dict(timeh={'$in':[24,48,72]},
         name={'$in':list(CH_CAS1.name)})
Q['best_fit.model']={'$ne':'cnst'}
#Q['cr_info.gt_3bmad']=1

for CR in HCI.heprn_cr.find(Q):
    X = pd.DataFrame(CR['cr_data'])
    # Only if there are 3 or more points above bmad
    #if (X.resp.abs()>1).sum()>=1: 
    RES.append(CR['best_fit'])

HepRn_raw = pd.DataFrame(RES)
HepRn_filt1 = HepRn_raw[(HepRn_raw.ga>=-0.7) & 
                        (HepRn_raw.ga<=2) & 
                        (HepRn_raw.ft_ch.isin(FT_CH))]
HepRn_filt1.insert(10,'ac50',10**HepRn_filt1.ga)

len(HepRn_filt1.name.unique())

51

### Filter by hit quality

In [5]:
X1=pd.DataFrame(list(HCI.heprn_hitqual.find({},dict(_id=0))))[['name','timeh','ft_ch','hit','quality_flags']]\
    .drop_duplicates().rename(columns=dict(hit='hit_call'))
HepRn_qc = HepRn_filt1.merge(X1[X1.hit_call==1],on=['name','timeh','ft_ch'])


### HepRn AC50 data

In [6]:
from box import Box
HEPRN = [Box(AC50=HepRn_filt1,potency_meth='tcpl:ac50'),
         Box(AC50=HepRn_qc,potency_meth='tcpl:ac50:manual_qc')]

### Summarize HepRn data

In [7]:
HCI.potency.rename('potency_0')

{'ok': 1.0}

In [8]:
def _p05(X):  return np.percentile(X,5)
def _p95(X): return np.percentile(X,95)
def _p50(X): return np.percentile(X,50)
def _n(X): return len(X)


for D in HEPRN:
    D.AC50 = D.AC50.merge(CH_CAS0[['dsstox_sid','name']],on='name')
    
    HepRn = D.AC50.groupby(['dsstox_sid','name'])\
                .aggregate(dict(ac50=[_p05,_p50,_p95]))
    HepRn.columns=[i+j for i,j in HepRn.columns]
    HepRn = HepRn.reset_index()
    HepRn_L = HepRn.melt(['dsstox_sid','name'],
                         var_name='potency_name',value_name='potency_val')
    HepRn_L.insert(0,'assay','HepRn')
    HepRn_L.insert(0,'platform','APR:HepRn')
    HepRn_L.insert(0,'ft','All')
    HepRn_L.insert(0,'timeh',72)


    HepRnTm = D.AC50.groupby(['dsstox_sid','name','timeh'])\
                .aggregate(dict(ac50=[_p05,_p50,_p95]))
    HepRnTm.columns=[i+j for i,j in HepRnTm.columns]
    HepRnTm = HepRnTm.reset_index()


    HepRnTm_L = HepRnTm.melt(['dsstox_sid','name','timeh'],
                         var_name='potency_name',value_name='potency_val')
    HepRnTm_L.insert(0,'assay','HepRn')
    HepRnTm_L.insert(0,'platform','APR:HepRn:Tm')
    HepRnTm_L.insert(0,'ft','All')

    HepRnFtTm = D.AC50.groupby(['dsstox_sid','name','ft','timeh'])\
                .aggregate(dict(ac50=[_p05,_p50,_p95]))
    HepRnFtTm.columns=[i+j for i,j in HepRnFtTm.columns]
    HepRnFtTm = HepRnFtTm.reset_index()

    HepRnFtTm_L = HepRnFtTm.melt(['dsstox_sid','name','ft','timeh'],
                         var_name='potency_name',value_name='potency_val')
    HepRnFtTm_L.insert(0,'assay','HepRn')
    HepRnFtTm_L.insert(0,'platform','APR:HepRn:Tm:Ft')
    
    D.potency = pd.concat((HepRnTm_L,HepRn_L,HepRnFtTm_L))
    D.potency['days_ivt'] = D.potency.timeh/24
    D.potency.insert(D.potency.shape[1],'potency_meth',D.potency_meth)

In [9]:
for i in (HepRnTm_L,HepRn_L,HepRnFtTm_L): del(i)

### Store final HepRn hits

In [14]:

FT_hits = D.potency.columns

HCI.potency.delete_many(dict(assay='HepRn'))

for D in HEPRN:
    print(D.potency_meth,D.potency.shape)
    HCI.potency.insert_many(D.potency.to_dict('records'))
#assert HCI.potency.count_documents(dict(assay='HepRnTm',platform='APR'))==X1.shape[0]

tcpl:ac50 (2157, 10)
tcpl:ac50:manual_qc (990, 10)


## ToxCast

The ToxCast hits are compiled as follows
* See heprn-toxcast-v1a for hits and results are stoed in  This file contains the assay level hits along with qc flags from invitrodb v3.  
* The final hits are identified based on quality and the results are stored in 
mongodb://pb.epa.gov/heprn_v1/potency

### Hits

In [15]:
EXC_FITS = ['Only highest conc above baseline, active',
            'Only one conc above baseline, active']
Tech=['NVS', 'ATG', 'BSK', 'CLD', 'TOX21', 'APR','ACEA','CEETOX', 'OT']

ToxCast_raw = pd.read_csv(DAT_DIR+'heprn-toxcast-hits-qual.csv')

ToxCast_filt1= ToxCast_raw.loc[(ToxCast_raw.modl_ga>-0.4) & \
                             (ToxCast_raw.modl_ga<2.3) & 
                            (ToxCast_raw.technology.isin(Tech))]

ToxCast_filt1.insert(10,'ac50',10**ToxCast_filt1.modl_ga)
ToxCast_filt1.rename(columns=dict(chemical='name'),inplace=True)
ToxCast_qc = ToxCast_filt1.loc[~(ToxCast_filt1.flag.isin(EXC_FITS))]



In [16]:
from box import Box
TOXCAST = [Box(AC50=ToxCast_filt1[ToxCast_filt1.dsstox_sid.isin(CH_CAS1.dsstox_sid)],potency_meth='tcpl:ac50'),
           Box(AC50=ToxCast_qc[ToxCast_qc.dsstox_sid.isin(CH_CAS1.dsstox_sid)],potency_meth='tcpl:ac50:manual_qc')]

### Summarize ToxCast assay data

#### All ToxCast assays

In [17]:
def _p05(X):  return np.percentile(X,5)
def _p95(X): return np.percentile(X,95)
def _p50(X): return np.percentile(X,50)
def _n(X): return len(X)


for D in TOXCAST:
    print(D.potency_meth)
    ToxCast = D.AC50.groupby(['dsstox_sid','name'])\
                   .aggregate(dict(ac50=[_p05,_p50,_p95]))
    ToxCast.columns=[i+j for i,j in ToxCast.columns]
    ToxCast = ToxCast.reset_index()
    ToxCast_L = ToxCast.melt(['dsstox_sid','name'],
                         var_name='potency_name',value_name='potency_val')
    ToxCast_L.insert(ToxCast_L.shape[1],'assay','ToxCast')
    ToxCast_L.insert(ToxCast_L.shape[1],'platform','All')
    ToxCast_L.insert(0,'timeh',24)
    ToxCast_L.insert(0,'ft','All')

    ToxCastTch = D.AC50.groupby(['dsstox_sid','name','technology'])\
                   .aggregate(dict(ac50=[_p05,_p50,_p95]))
    ToxCastTch.columns=[i+j for i,j in ToxCastTch.columns]
    ToxCastTch = ToxCastTch.reset_index()
    ToxCastTch_L = ToxCastTch.melt(['dsstox_sid','name','technology'],
                         var_name='potency_name',value_name='potency_val')\
                    .rename(columns=dict(technology='platform'))
    ToxCastTch_L.insert(ToxCastTch_L.shape[1],'assay','ToxCast')
    ToxCastTch_L.insert(0,'timeh',24)
    ToxCastTch_L.insert(0,'ft','All')

    ToxCastTchFt = D.AC50.groupby(['dsstox_sid','name','technology','aenm'])\
                   .aggregate(dict(ac50=[_p05,_p50,_p95]))
    ToxCastTchFt.columns=[i+j for i,j in ToxCastTchFt.columns]
    ToxCastTchFt = ToxCastTchFt.reset_index()
    ToxCastTchFt_L = ToxCastTchFt.melt(['dsstox_sid','name','technology','aenm'],
                         var_name='potency_name',value_name='potency_val')\
                    .rename(columns=dict(technology='platform',
                                         aenm='ft'))
    ToxCastTchFt_L.insert(ToxCastTchFt_L.shape[1],'assay','ToxCast')
    ToxCastTchFt_L.insert(0,'timeh',24)
    
    Tx_HepRn = D.AC50[D.AC50.cell=='Hepat'].groupby(['dsstox_sid','name'])\
                   .aggregate(dict(ac50=[_p05,_p50,_p95]))
    Tx_HepRn.columns=[i+j for i,j in Tx_HepRn.columns]
    Tx_HepRn = Tx_HepRn.reset_index()
    Tx_HepRn_L = Tx_HepRn.melt(['dsstox_sid','name'],
                         var_name='potency_name',value_name='potency_val')
    Tx_HepRn_L.insert(0,'assay','ToxCast')
    Tx_HepRn_L.insert(0,'technology','APR:HepRn')
    Tx_HepRn_L.insert(0,'timeh',24)
    Tx_HepRn_L.insert(0,'ft','All')

    Tx_HepRnTm = D.AC50[D.AC50.cell=='Hepat'].groupby(['dsstox_sid','name','timeh'])\
                   .aggregate(dict(ac50=[_p05,_p50,_p95]))
    Tx_HepRnTm.columns=[i+j for i,j in Tx_HepRnTm.columns]
    Tx_HepRnTm = Tx_HepRnTm.reset_index()
    Tx_HepRnTm.loc[:,'timeh'] = Tx_HepRnTm.timeh.astype(np.uint)
    Tx_HepRnTm_L = Tx_HepRnTm.melt(['dsstox_sid','name','timeh'],
                         var_name='potency_name',value_name='potency_val')\
                    .rename(columns=dict(technology='platform'))
    Tx_HepRnTm_L.insert(0,'assay','ToxCast')
    Tx_HepRnTm_L.insert(0,'technology','APR:HepRnTm')
    Tx_HepRnTm_L.insert(0,'ft','All')
    
    D.potency = pd.concat((ToxCast_L,ToxCastTch_L,Tx_HepRn_L,Tx_HepRnTm_L,ToxCastTchFt_L))
    D.potency['days_ivt'] = D.potency.timeh/24
    D.potency.insert(D.potency.shape[1],'potency_meth',D.potency_meth)

tcpl:ac50
tcpl:ac50:manual_qc


#### Store Final ToxCast Hits 

In [18]:

HCI.potency.delete_many(dict(assay='ToxCast'))

for D in TOXCAST:
    print(D.potency_meth,D.potency.shape)
    HCI.potency.insert_many(D.potency[FT_hits].to_dict('records'))


tcpl:ac50 (12291, 11)
tcpl:ac50:manual_qc (11751, 11)


In [19]:
for i in [ToxCast_L,ToxCastTch_L,Tx_HepRn_L,Tx_HepRnTm_L,ToxCastTchFt_L]: del(i)

In [20]:
#assert HCI.potency.count_documents(dict(assay='HepRnTm',platform='APR'))==X1.shape[0]

## HepRnAtg

In [21]:
HEPRN_ATG=[]
HCI.potency.delete_many(dict(assay='HepRnAtg'))

for D1 in TOXCAST:
    print(D1.potency_meth)
    D2 = next(i for i in HEPRN if i.potency_meth==D1.potency_meth)
    X1 = D1.AC50[D1.AC50.technology=='ATG'][['dsstox_sid','name','ac50']]
    R = []
    for th in [24,48,72]:
        X = X1.copy()
        X.insert(0,'timeh',th)
        R.append(X)
    Atg = pd.concat(R)
    
    HepRn= D2.AC50[['dsstox_sid','name','ac50','timeh']]
    
    HepRnAtg = pd.concat((Atg,HepRn)).groupby(['dsstox_sid','name','timeh'])\
                .aggregate(dict(ac50=[_p05,_p50,_p95]))
    HepRnAtg.columns=[i+j for i,j in HepRnAtg.columns]
    HepRnAtg = HepRnAtg.reset_index()


    HepRnAtg_L = HepRnAtg.melt(['dsstox_sid','name','timeh'],
                          var_name='potency_name',value_name='potency_val')
    HepRnAtg_L.insert(0,'assay','HepRnAtg')
    HepRnAtg_L.insert(0,'platform','APR:HepRn:Tm|ToxCast:ATG')
    HepRnAtg_L.insert(0,'ft','All')
    HepRnAtg_L.insert(0,'days_ivt',HepRnAtg_L.timeh/24)
    HepRnAtg_L.insert(0,'potency_meth',D1.potency_meth)
    
        
    HCI.potency.insert_many(HepRnAtg_L.to_dict('records'))

tcpl:ac50
tcpl:ac50:manual_qc


# IVIV extrapolation

## Parallel machine

In [44]:
import os
import ipyparallel as PP

RC=PP.Client(profile='pb_parallel')
d_view=RC[:]
lb_view = RC.load_balanced_view()
lb_view.block = True
x=open("../env3.py",'r').read()
d_view.execute(x)

<AsyncResult: execute>

## Tasks

In [117]:
WORK0 = []

CH_tk = HCI.heprn_iviv.distinct('name')
for R in HCI.potency.find(dict(name={'$in':CH_tk},
                               #assay={'$in':['HepRnAtg']}
                              ),
                          dict(_id=0)):
    R.update(dict(days_ivv=[90,365*2]))
    WORK0.append(R)
len(WORK0)

28104

In [270]:
# from tk.httk import *
# HCI.potency_aed.drop()
# HCI.potency_error.drop()

# w=WORK0[0]
# runCalcAllAED(dbc_iviv=HCI.heprn_iviv,dbc_sav=HCI.potency_aed,
#               dbc_err=HCI.potency_error,**w)

In [115]:
import datetime
HCI.potency_aed.drop()
HCI.potency_error.drop()

print("Start ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK0))

RES = lb_view.map(lambda x: runCalcAllAED(dbc_iviv=HCI.heprn_iviv,
                                          dbc_sav=HCI.potency_aed,
                                          dbc_err=HCI.potency_error,**x),
                  WORK0)

print("End ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK0))

Start ...  2019-08-30 00:08:37 tasks =  915
End  2019-08-30 00:09:27 tasks =  915


In [116]:
HCI.potency_error.count_documents({})

0

In [97]:
HCI.potency_aed.create_index('assay')
HCI.potency_aed.create_index('dsstox_sid')
HCI.potency_aed.create_index('name')
HCI.potency_aed.create_index('study_type')
HCI.potency_aed.create_index('days_ivv')


'days_ivv_1'

In [119]:
HCI.potency_aed.distinct('potency_meth')

['tcpl:ac50', 'tcpl:ac50:manual_qc']

## Load liver effect data 

In [81]:
LIV_EFF = pd.DataFrame(list(HCI.toxref_rat_v2.find({},dict(_id=0))))
LIV_EFF.shape

(13227, 57)

## Effect clean-up 

In [84]:
HCI.rat_effects_v2.drop()
X = LIV_EFF
Y = X[X.casrn.isin(CH_CAS1.casrn)]
Y = Y[(Y.pod_type=='loael') & (Y.study_type.isin(['SUB','CHR']))]
Y.effect_desc.replace({'[other]':'other',
                       'nos':'other',
                       'lesion(s) (nos)':'other',
                       'accentuated lobular pattern':'other',
                       'mineralization':'other',
                       'discolored':'other',
                       'pigmentation':'other',
                       'absolute':'hepatomegaly',
                       'pigment':'other',
                       'enlarged':'hepatomegaly',
                       'vacuolization cytoplasmic':'vacuolization',
                       'hematopoietic cell proliferation erythrocytic':'hematopoietic cell prolif.',
                       'hematopoietic cell proliferation': 'hematopoietic cell prolif.',
                       'relative to brain weight':'hepatomegaly',
                       'liver wt change (rel)':'hepatomegaly',
                       'relative to body weight': 'hepatomegaly',
                       'dilation':'other',
                       'foam cell aggregation':'fatty change',
                       'adenoma':'adenoma/carcinoma',
                       'hyperplastic nodule (pre-neoplastic)':'pre-neoplastic nodule',
                       'carcinoma':'adenoma/carcinoma',
                       'spongiosis hepatis':'spongiosis heptis/cyst',
                       'cyst':'spongiosis heptis/cyst',
                       'adenoma/carcinoma combined':'adenoma/carcinoma'
                      },
                     inplace=True)

Y.effect_desc.replace({k:k.capitalize() for k in Y.effect_desc.unique()},
                      inplace=True)
Y.rename(columns={'preferred_name':'name',
                  'dsstox_substance_id':'dsstox_sid'},inplace=True)

LIV_EFF1 = Y.copy()
print(Y.shape)
HCI.rat_effects_v2.insert_many(LIV_EFF1.to_dict('records'))

(276, 57)


## POD Summary

In [66]:
def _p05(X):  return np.percentile(X,5)
def _p95(X): return np.percentile(X,95)
def _p50(X): return np.percentile(X,50)

Dur = dict(SUB=90,CHR=365*2)

TOX_POD = LIV_EFF1.groupby(['name','dsstox_sid','casrn','study_type'])\
            .aggregate(dict(pod_value=[_p05,_p50,_p95]))
TOX_POD.columns = ['loael'+j for i,j in TOX_POD.columns]

X = TOX_POD.reset_index()
TOX_POD_L = X[X.study_type.isin(Dur.keys())]\
                .melt(['name','dsstox_sid','casrn','study_type'],
                      var_name='pod_name',value_name='pod_value')

TOX_POD_L.insert(5,'days_ivv',TOX_POD_L.study_type.apply(lambda i: Dur[i]))
TOX_POD_L.insert(6,'effect','All')


## POD Lesion Summary

In [67]:
def _p05(X):  return np.percentile(X,5)
def _p95(X): return np.percentile(X,95)
def _p50(X): return np.percentile(X,50)

Dur = dict(SUB=90,CHR=365*2)

TOX_POD_EF = LIV_EFF1.groupby(['name','dsstox_sid','casrn','study_type','effect_desc'])\
                     .aggregate(dict(pod_value=[_p05,_p50,_p95]))
TOX_POD_EF.columns = ['loael'+j for i,j in TOX_POD_EF.columns]

X = TOX_POD_EF.reset_index()
TOX_POD_EF_L = X[X.study_type.isin(Dur.keys())]\
                .melt(['name','dsstox_sid','casrn','study_type','effect_desc'],
                      var_name='pod_name',value_name='pod_value')\
                .rename(columns=dict(effect_desc='effect'))
TOX_POD_EF_L.insert(5,'days_ivv',TOX_POD_EF_L.study_type.apply(lambda i: Dur[i]))

## Store POD Data

In [68]:
HCI.rat_pod_v2.drop()
HCI.rat_pod_v2.insert_many(TOX_POD_L.to_dict('records'))
HCI.rat_pod_v2.insert_many(TOX_POD_EF_L[TOX_POD_L.columns].to_dict('records'))

# IVIVE vs POD

## Merge AED and POD data

In [120]:
HCI.potency_aed_pod.drop()

TOX_POD = pd.DataFrame(HCI.rat_pod_v2.find({},dict(_id=0)))

for (name,dsstox_sid,days_ivv),Xi in TOX_POD.groupby(['name','dsstox_sid','days_ivv']):
    AEDi = pd.DataFrame(HCI.potency_aed.find(dict(dsstox_sid=dsstox_sid,days_ivv=int(days_ivv)),
                                                  dict(_id=0)))\
             .merge(Xi,on=['name','days_ivv','dsstox_sid','casrn'])
    print(name,days_ivv,AEDi.shape)
    if AEDi.shape[0]>0:
        HCI.potency_aed_pod.insert_many(AEDi.to_dict('records'))
    
del(AEDi)
del(TOX_POD)

2,4-Dichlorophenoxyacetic acid 90 (17712, 18)
2,4-Dichlorophenoxyacetic acid 730 (13284, 18)
Acetaminophen 90 (21384, 18)
Acetochlor 90 (54864, 18)
Acetochlor 730 (82296, 18)
Alachlor 730 (84240, 18)
Ametryn 730 (26568, 18)
Bensulide 90 (174528, 18)
Bensulide 730 (261792, 18)
Bentazon 90 (0, 18)
Boscalid 90 (37260, 18)
Boscalid 730 (37260, 18)
Carbaryl 730 (98064, 18)
Chlorpropham 90 (45036, 18)
Chlorpropham 730 (45036, 18)
Cyclanilide 90 (41472, 18)
Cyproconazole 90 (74520, 18)
Cyprodinil 90 (215460, 18)
Cyprodinil 730 (61560, 18)
Di(2-ethylhexyl) phthalate 90 (43092, 18)
Di(2-ethylhexyl) phthalate 730 (43092, 18)
Dichlobenil 90 (11664, 18)
Dichlobenil 730 (34992, 18)
Dimethenamid 90 (42768, 18)
Dimethenamid 730 (28512, 18)
Diphenylamine 90 (59400, 18)
Diphenylamine 730 (35640, 18)
Dithiopyr 90 (74196, 18)
Dithiopyr 730 (49464, 18)
Diuron 730 (52488, 18)
Ethofumesate 90 (46656, 18)
Ethofumesate 730 (46656, 18)
Etoxazole 90 (100440, 18)
Etoxazole 730 (133920, 18)
Fenarimol 730 (66312, 

In [121]:
HCI.potency_aed_pod.create_index('dsstox_sid')
HCI.potency_aed_pod.create_index('casrn')
HCI.potency_aed_pod.create_index('name')
HCI.potency_aed_pod.create_index('study_type')
HCI.potency_aed_pod.create_index('days_ivv')
HCI.potency_aed_pod.create_index('days_ivt')
HCI.potency_aed_pod.create_index('dose_metric')
HCI.potency_aed_pod.create_index('ivive_method')
HCI.potency_aed_pod.create_index('potency_meth')
HCI.potency_aed_pod.create_index('potency_name')
HCI.potency_aed_pod.create_index('pod_name')
HCI.potency_aed_pod.create_index('assay')
HCI.potency_aed_pod.create_index('platform')


'platform_1'

In [122]:
#HCI.potency_aed_pod.distinct('assay')
#HCI.potency_aed_pod.count_documents(dict(assay='HepRnAtg'))

## Compare AED and POD

### Performance metrics

In [123]:
# IVV= ['loael_p05','loael_p50','loael_p95']
# IVT= ['aed_p05','aed_p50','aed_p95']
# DM = ['auc','cmax','auc']

# y = POD_tox
# x = POD_name

def calc_SSTotal(x,**kws):        
    return ((x-x.mean())**2).sum()


def calc_RMSE(x,y,**kws):
    return np.sqrt(calc_Resid(x,y))

def calc_R2(x,y,log_trans=False,**kws):    
    r2 = 1-(calc_Resid(x,y)/calc_SSTotal(x))
    if log_trans:
        r2 = 10**r2
    return r2


def calc_pearson(x,y,**kws):
    return sp.stats.pearsonr(x,y)[0]

def calc_spearman(x,y,**kws):
    return sp.stats.spearmanr(x,y)[0]

def calc_spearman_p(x,y,**kws):
    return sp.stats.spearmanr(x,y)[1]

def calc_MeanFold_yx(x,y,log_trans=False,**kws):
    """Mean(y/x)"""
    fc= np.mean(y-x)    
    if log_trans:
        fc = 10**fc
    return fc

def calc_LPR(x,y,**kws):
    """mean(y-x)"""
    return np.mean(y-x)

def calc_MeanFold_xy(x,y,log_trans=False,**kws):
    fc= np.mean(x-y)    
    if log_trans:
        fc = 10**fc
    return fc    

def calc_Overpred(x,y,**kws):
    """Fraction """
    return  1.0*np.sum(y<x)/len(x)

def calc_Underpred(x,y,**kws):
    return  1.0*np.sum(y>x)/len(x)

def calc_WithinLog10(x,y,**kws):
    return  1.0*np.sum(np.abs(x-y)<=1)/len(x)

def calc_BeyondLog10(x,y,**kws):
    return  1.0*np.sum(np.abs(x-y)>1)/len(x)

def calc_n(x,y,**kws):
    return len(x)

def calc_Underpred_MeanFold_xy(x,y,**kws):
    return calc_MeanFold_xy(x[x<y],y[x<y],**kws)

def calc_Overpred_MeanFold_yx(x,y,**kws):
    return calc_MeanFold_yx(x[x>y],y[x>y],**kws)

    
Summary_F = dict(RMSE=calc_RMSE,
                 R2=calc_R2,
                 resid=calc_Resid,
                 R_p=calc_pearson,
                 R_s=calc_spearman,
                 p_s=calc_spearman_p,
                 PR=calc_MeanFold_yx,
                 LPR = calc_LPR,
                 n_chem=calc_n,
                 LPR_lt_1=calc_WithinLog10,
                 LPR_gt_l=calc_BeyondLog10,
                 Underpred=calc_Underpred,
                 Overpred=calc_Overpred,
                 Underpred_PR=calc_Underpred_MeanFold_xy,
                 Overpred_PR=calc_Overpred_MeanFold_yx
                 )



### Evaluate

In [124]:
HCI.performance.drop()

FX = ['study_type', 'assay','platform','ft','ivive_method','potency_meth',u'dose_metric',
      'potency_name','pod_name','ft','effect','days_ivt']

for study in ['SUB','CHR']:
    for assay in ['HepRn','HepRnAtg','ToxCast']:
        for platform in HCI.potency_aed_pod.find(dict(study_type=study,assay=assay)).distinct('platform'):
            if platform not in ['APR:HepRn:Tm','APR:HepRn','APR:HepRn:Tm:Ft',
                                 'APR:HepRn:Tm|ToxCast:ATG','All']: 
                continue
            X = pd.DataFrame(HCI.potency_aed_pod.find(dict(platform=platform,
                                                           study_type=study,
                                                           assay=assay,
                                                           pod_name='loael_p05',
                                                           potency_name='ac50_p05'
                                                          )))
            print(study,assay,platform,X.shape[0])
            RES=[]
            for F_i,X_i in X.groupby(FX):
                if X_i.shape[0]<2: continue
                R = dict(zip(FX,F_i))
                R.update({k:f(np.log10(X_i['aed']),np.log10(X_i['pod_value']),
                              log_trans=True) for k,f in Summary_F.items()})

                HCI.performance.insert_one(R)
            

SUB HepRn APR:HepRn:Tm 6732
SUB HepRn APR:HepRn 2640
SUB HepRn APR:HepRn:Tm:Ft 20028


/home/ishah/.conda/envs/py37a/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


SUB HepRnAtg APR:HepRn:Tm|ToxCast:ATG 8784
SUB ToxCast All 2928
CHR HepRn APR:HepRn:Tm 8316
CHR HepRn APR:HepRn 3228
CHR HepRn APR:HepRn:Tm:Ft 26280
CHR HepRnAtg APR:HepRn:Tm|ToxCast:ATG 10200
CHR ToxCast All 3408


In [126]:
#HCI.performance.drop()

FX = ['study_type','assay','platform','ft','ivive_method','potency_meth',u'dose_metric',
      'potency_name','pod_name','ft','effect','days_ivt']

for study in ['SUB','CHR']:
    for assay in ['ToxCast']:
        for platform in HCI.potency_aed_pod.find(dict(study_type=study,assay=assay)).distinct('platform'):
            if platform in ['APR:HepRn:Tm','APR:HepRn','APR:HepRn:Tm:Ft',
                            'APR:HepRn:Tm|ToxCast:ATG','All']: 
                continue
            X = pd.DataFrame(HCI.potency_aed_pod.find(dict(platform=platform,
                                                           study_type=study,
                                                           assay=assay,
                                                           pod_name='loael_p05',
                                                           potency_name='ac50_p05'
                                                          )))
            print(study,assay,platform,X.shape[0])
            RES=[]
            for F_i,X_i in X.groupby(FX):
                if X_i.shape[0]<2: continue
                R = dict(zip(FX,F_i))
                R.update({k:f(np.log10(X_i['aed']),np.log10(X_i['pod_value']),
                              log_trans=True) for k,f in Summary_F.items()})
                HCI.performance.insert_one(R)
            

SUB ToxCast BSK 60456
SUB ToxCast CLD 21228
SUB ToxCast ATG 51732
SUB ToxCast NVS 43908
SUB ToxCast APR 22416
SUB ToxCast ACEA 9324
SUB ToxCast OT 8940
SUB ToxCast nan 4464
SUB ToxCast CEETOX 4920
SUB ToxCast TOX21 96
CHR ToxCast BSK 77136
CHR ToxCast CLD 25716
CHR ToxCast ATG 64668
CHR ToxCast NVS 50664
CHR ToxCast APR 32832
CHR ToxCast ACEA 10800
CHR ToxCast OT 10368
CHR ToxCast nan 6552
CHR ToxCast CEETOX 6984
CHR ToxCast TOX21 4776


In [125]:
for i in ['study_type','assay','platform','days_ivt','days_ivv','potency_name','pod_name',
          'ivive_method','potency_meth','ft','dose_metric']:
    print(i)
    HCI.performance.create_index(i)

study_type
assay
platform
days_ivt
days_ivv
potency_name
pod_name
ivive_method
potency_meth
ft
dose_metric
